In [ ]:
#default_exp script

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.notebook.showdoc import show_doc

from argparse import ArgumentParser

In [ ]:
#export
_all_ = ['Param']

## call_parse

In [ ]:
#export
def _param_pre(self): return '--' if self.opt else ''
def _param_kwargs(self): return {k:v for k,v in self.__dict__.items() if v is not None and k!='opt'}

In [ ]:
#export
mk_class('Param', help=None, type=None, opt=True, action=None, nargs=None, const=None, choices=None, required=None,
         pre=property(_param_pre), kwargs=property(_param_kwargs),
         doc="A parameter in a function used in `anno_parser` or `call_parse`")

In [ ]:
show_doc(Param, title_level=3)

In [ ]:
p = Param(opt=True, help=1)
p.kwargs

In [ ]:
#export
def anno_parser(func):
    "Look at params (annotated with `Param`) in func and return an `ArgumentParser`"
    p = ArgumentParser(description=func.__doc__)
    for k,v in inspect.signature(func).parameters.items():
        param = func.__annotations__.get(k, Param())
        kwargs = param.kwargs
        if v.default != inspect.Parameter.empty: kwargs['default'] = v.default
        p.add_argument(f"{param.pre}{k}", **kwargs)
    return p

In [ ]:
def f1(p1:Param("h1", str)='1', p2:Param("h2", float)=1e-3):
    "mydoc"
    pass

t = anno_parser(f1)
h = t.parse_args(['--p1', 'a'])
test_eq(h.p1, 'a')
test_eq(h.p2, 1e-3)

In [ ]:
#export
def call_parse(func):
    "Decorator to create a simple CLI from `func` using `anno_parser`"
    name = inspect.currentframe().f_back.f_globals['__name__']
    if name == "__main__":
        args = anno_parser(func).parse_args()
        func(**args.__dict__)
    else: return func

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)